In [ ]:
## print('Starting generic python imports')
from pathlib import Path
import numpy as np
import os
from time import time, sleep
import sys
from munch import Munch

print('Doing FKMC imports')
from FKMC.montecarlo import *

## overall input parameters
original_repeats = 4
original_Ts = np.linspace(0.1, 4, 200)

start_point = 0
end_point = 100 ##to do them all this should be 799 but I'll do it in stages

repeats = 4 * 10 #using 1e5 instead of 1e6 steps so here using a factor of 10 more repeats
Ts = original_Ts[start_point//original_repeats:end_point//original_repeats]
print(f'T from {min(Ts)} to {max(Ts)}')

J = 5
U = 5
alpha = 1.25
total_jobs = len(Ts) * repeats

initial_states_cycle = [
    lambda N: np.arange(N, dtype = np.float64) % 2,
    lambda N: (np.arange(N, dtype = np.float64)+1) % 2,
    lambda N: np.zeros(N, dtype = np.float64),
    lambda N: np.ones(N, dtype = np.float64),
]

#give information to the dispatch script
batch_params = Munch(total_jobs = total_jobs, repeats = repeats, Ts = Ts, structure_names = ('repeats', 'Ts'))

print('Getting environment variables')
slurm_job_id = int(os.getenv('SLURM_ARRAY_JOB_ID', -1))
slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_ID', 29))
max_slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_MAX', total_jobs - 1))
working_dir = Path(".")

##sleep if necessary3-400
debug = (slurm_job_id == -1)
sleeptime = np.random.random() * (30)
print(f'Waiting for {sleeptime:.0f} seconds to randomise the finish time')
if not debug: sleep(sleeptime)


##using the value of slurm_task_id, decide what parameters this job will have
T_i = slurm_task_id // repeats
T = Ts[T_i]
repeat_i = slurm_task_id % repeats
make_initial_state = initial_states_cycle[repeat_i % len(initial_states_cycle)]

print(f'T_i = {T_i}, T = {T}, repeat_i = {repeat_i}')

print(f'total_jobs = {total_jobs} max_slurm_task_id = {max_slurm_task_id}')

Ns = [4,8,16,32,64,128]
logs = np.empty(shape = len(Ns), dtype = object)

thousand = 1000 if not debug else 1
for i, N in enumerate(Ns):
    parameters = dict(t = 1, alpha = alpha, mu = 0, beta = 1/T, J = J, U = U, normalise = True)
    MCMC_params = dict(
            state = make_initial_state(N),
            N_steps = int(1e5) if not debug else 100,
            N_burn_in = int(1e3) if not debug else 0,
            thin = 1000 if not debug else 1,
            logger = Eigenspectrum_IPR_all(bins = 2000, limit = 5),
            proposal = p_multi_site_uniform_reflect,
            accept_function = perturbation_accept,
            warnings = True,
        )

    print(f'starting N = {N}')
    t0 = time()
    logs[i] = FK_mcmc(**MCMC_params, parameters = parameters)
    logs[i].time = time() - t0

    parameters['J_matrix'] = '...'
    MCMC_params['state'] = '...'
    print(f'''
    params = {parameters},
    mc_params = {MCMC_params}
    ''')

filename = f'{slurm_task_id}.npz'

print(f'{time() - t0:.0f} seconds, saving in {filename}')

if slurm_job_id != -1: np.savez_compressed(working_dir/filename, 
        Ns = Ns, Ts = Ts, parameters = parameters, MCMC_params = MCMC_params, logs = logs, allow_pickle = True,
        desc = ''
        )
print('done')

In [2]:
"""
if slurm_job_id == -1:
    from matplotlib import pyplot as plt
    import logging
    mpl_logger = logging.getLogger('matplotlib')
    mpl_logger.setLevel(logging.WARNING) 
    %matplotlib inline
    
    f, ax = plt.subplots(figsize = (20, 5))
    ax.scatter(Ts, 0.3*np.ones_like(Ts), marker = '|')
    ax.scatter(original_Ts, 0.32*np.ones_like(original_Ts), marker = '|')
    ax.set(ylim = (0,1))
"""

"\nif slurm_job_id == -1:\n    from matplotlib import pyplot as plt\n    import logging\n    mpl_logger = logging.getLogger('matplotlib')\n    mpl_logger.setLevel(logging.WARNING) \n    %matplotlib inline\n    \n    f, ax = plt.subplots(figsize = (20, 5))\n    ax.scatter(Ts, 0.3*np.ones_like(Ts), marker = '|')\n    ax.scatter(original_Ts, 0.32*np.ones_like(original_Ts), marker = '|')\n    ax.set(ylim = (0,1))\n"